# ACS data

Let's start by reading the data!

In [21]:
import pandas as pd
demo = pd.read_csv(r'..\data\raw\nyc_acs_demographics.csv')
demo.head()

,Unnamed: 0,geoid,acs_demog_var,value
0,0,360050001001,total_pop,7080.0
1,1,360050001001,two_or_more_races_pop,41.0
2,2,360050001001,vacant_housing_units_for_rent,0.0
3,3,360050001001,unemployed_pop,0.0
4,4,360050001001,black_pop,3984.0


The variables are stacked in the acs_demog_var column, we should unpack to different columns.

In [22]:
demo = acs.pivot(index='geoid', columns='acs_demog_var', values='value')
demo.describe()

acs_demog_var,aggregate_travel_time_to_work,amerindian_pop,asian_pop,black_pop,commuters_16_over,commuters_by_public_transportation,employed_pop,families_with_young_children,family_households,female_pop,...,other_race_pop,owner_occupied_housing_units_median_value,pop_16_over,renter_occupied_housing_units_paying_cash_median_gross_rent,total_pop,two_or_more_races_pop,unemployed_pop,vacant_housing_units,vacant_housing_units_for_rent,white_pop
count,362.000000,6339.000000,6339.000000,6339.000000,6339.000000,6339.000000,6339.000000,6339.000000,6339.000000,6339.000000,...,6339.000000,4.556000e+03,6339.000000,5807.000000,6339.000000,6339.000000,6339.000000,6339.000000,6339.000000,6339.000000
mean,34616.422652,2.380186,185.575170,295.618394,602.898880,352.085029,644.844613,98.536362,300.488405,702.815271,...,11.458116,6.872881e+05,1091.115160,1523.152574,1343.245149,25.833728,47.946364,50.476100,12.062786,432.200663
std,20381.770030,13.583396,285.281508,436.053112,336.115082,248.154204,356.828817,87.249059,158.296530,358.603710,...,43.207546,3.830040e+05,539.026034,535.556652,658.446589,43.511631,47.769236,68.955807,24.462099,486.881898
min,6450.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,9.999000e+03,0.000000,229.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,21785.000000,0.000000,10.000000,7.000000,397.000000,191.000000,425.000000,39.000000,200.000000,472.000000,...,0.000000,4.430000e+05,747.000000,1232.000000,918.000000,0.000000,15.000000,11.000000,0.000000,46.000000
50%,30877.500000,0.000000,72.000000,79.000000,552.000000,310.000000,588.000000,79.000000,278.000000,648.000000,...,0.000000,5.981500e+05,1013.000000,1442.000000,1249.000000,9.000000,36.000000,34.000000,0.000000,281.000000
75%,40933.750000,0.000000,237.500000,451.000000,751.500000,463.000000,800.000000,136.000000,374.000000,863.000000,...,0.000000,8.288000e+05,1332.000000,1728.000000,1648.000000,35.000000,67.000000,65.000000,17.000000,674.000000
max,195980.000000,274.000000,3960.000000,4744.000000,6483.000000,4849.000000,6767.000000,1030.000000,2250.000000,5037.000000,...,721.000000,2.000001e+06,7881.000000,3501.000000,8830.000000,766.000000,575.000000,1513.000000,467.000000,6111.000000


In [23]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6339 entries, 360050001001 to 361190055004
Data columns (total 33 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   aggregate_travel_time_to_work                                362 non-null    float64
 1   amerindian_pop                                               6339 non-null   float64
 2   asian_pop                                                    6339 non-null   float64
 3   black_pop                                                    6339 non-null   float64
 4   commuters_16_over                                            6339 non-null   float64
 5   commuters_by_public_transportation                           6339 non-null   float64
 6   employed_pop                                                 6339 non-null   float64
 7   families_with_young_children                               

There is missing data in some of the columns

In [24]:
demo.columns[acs.isna().any()].tolist()

['aggregate_travel_time_to_work',
 'income_per_capita',
 'median_age',
 'median_income',
 'median_rent',
 'median_year_structure_built',
 'owner_occupied_housing_units_median_value',
 'renter_occupied_housing_units_paying_cash_median_gross_rent']

Different strategies can be used to fill this data, like using the mean of the total population. Since we have location data, it seems more interesting to fill with mean of the adjacent blocks. We'll exclude the aggregate_travel_time_to_work column since it's mostly empty,

In [25]:
import geopandas
import numpy as np
block = geopandas.read_file(r'..\data\raw\nyc_cbg_geoms.geojson')
block['geoid'] = block['geoid'].astype(np.int64)

In [26]:
demo = demo.drop(columns='aggregate_travel_time_to_work')
demo = block.merge(demo, on='geoid', how='inner')
demo = demo.set_index('geoid')

In [67]:
for index, geom in demo.iterrows():
    null_cols = geom[geom.isnull()].index.to_list()
    if null_cols:
        neighbors = demo[~demo.['geometry'].disjoint(geom['geometry'])].index.to_list()
        for col in null_cols:
            demo.at[index, col] = demo.loc[neighbors, col].mean()

In [76]:
print(acs.isna().sum(axis=0))
acs.columns[acs.isna().any()].tolist()

geometry                                                        0
amerindian_pop                                                  0
asian_pop                                                       0
black_pop                                                       0
commuters_16_over                                               0
commuters_by_public_transportation                              0
employed_pop                                                    0
families_with_young_children                                    0
family_households                                               0
female_pop                                                      0
hispanic_pop                                                    0
households                                                      0
housing_units                                                   0
housing_units_renter_occupied                                   0
income_per_capita                                               3
male_pop  

['income_per_capita',
 'median_age',
 'median_income',
 'median_rent',
 'median_year_structure_built',
 'owner_occupied_housing_units_median_value',
 'renter_occupied_housing_units_paying_cash_median_gross_rent']

There are still some missing values, we will fill them with the mean value later when we build the pipeline.